In [ ]:
import numpy as np
import pandas as pd
import pickle 
from IPython.display import Markdown, display
from typing import List
from joblib import Parallel, delayed
import time


from langchain.vectorstores import FAISS
import langchain_gigachat
from langchain_gigachat import GigaChat, GigaChatEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
 
def prepare_embs(emb):
    emb = np.array(emb)
    emb /= (((emb**2).sum(1))**0.5).reshape((emb.shape[0], 1))
    emb = emb.tolist()
    return emb

def prepare_emb(emb):
    emb = np.array(emb)
    emb /= ((emb**2).sum())**0.5
    emb = emb.tolist()
    return emb  

def display_markdown(text):
    display(Markdown(text.replace('Модели оценки кредитного риска', '')))

from langchain_gigachat import GigaChat, GigaChatEmbeddings

llm = GigaChat(
            model= 'GigaChat-Pro',
    )


llm_max = GigaChat(
            model = 'GigaChat-2-Max',
)


llm_max_strict = GigaChat(
            model = 'GigaChat-2-Max',
)


llm_deepseek_strict = GigaChat(
            model = 'DeepSeek-R1',
)


llm_deepseek = GigaChat(
            model = 'DeepSeek-R1',
)

### Читаем уже созданные чанки и вопросы


In [ ]:
db = FAISS.load_local(data_path,
                             None,
                             allow_dangerous_deserialization=True)

In [ ]:
docs = db.docstore._dict

In [ ]:
db.index.ntotal

In [ ]:
docs_keys = list(docs.keys())
docs[docs_keys[0]].page_content

In [ ]:
docs_texts = [docs[key].page_content for key in docs_keys]
docs_metas = [docs[key].metadata for key in docs_keys]
len(docs_texts), len(set(docs_texts))

In [ ]:
# queries = pd.read_excel('Прошлые вопросы (RAG).xlsx')
queries = pd.read_excel('C:\\Users\\22589910\\Desktop\\diplom\\indexes\\Документы + вопросы с ответами. БЕНЧИ КУЧКА 4.xlsx')
print(len(queries))
queries = queries.dropna()
print(len(queries))

In [ ]:
for q in queries['Вопрос ']:
    print(q)

#### Создаем простые вопросы

In [ ]:


GEN_QUESTIONS_SYSTEM_PROMPT = """NDA"""


In [ ]:
CHECK_RELEVANCE_CHUNK = """NDA"""

In [ ]:
from pydantic import BaseModel, Field

class ContextEstimation(BaseModel):
"""NDA"""
    
class QuestionGenerationItem(BaseModel):
"""NDA"""

class MultipleQuestionGeneration(BaseModel):
"""NDA"""



In [ ]:
check_relevance_chat_template = ChatPromptTemplate.from_messages([
    ('system', CHECK_RELEVANCE_CHUNK),
    ('user', 'Отрывок, который необходимо оценить:\n{text}')
])

gen_questions_chat_template = ChatPromptTemplate.from_messages([
    ('system', GEN_QUESTIONS_SYSTEM_PROMPT),
    ('user', 'Отрывок, по которому необходимо составить вопросы:\n{text}')
])


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
gen_questions_chat_template.invoke({'text': 'Привт', 'sample_questions': 'ddd'})

In [ ]:
# check_relevance_pipe_deepseek = check_relevance_chat_template | llm_deepseek_strict | StrOutputParser()
# gen_questions_pipe = gen_questions_chat_template | llm_deepseek | StrOutputParser()

# check_relevance_pipe_deepseek = check_relevance_chat_template | llm_deepseek_strict.with_structured_output(ContextEstimation)
# gen_questions_pipe_deepseek = gen_questions_chat_template | llm_deepseek.with_structured_output(MultipleQuestionGeneration)

check_relevance_pipe = check_relevance_chat_template | llm_max_strict.with_structured_output(ContextEstimation)
gen_questions_pipe = gen_questions_chat_template | llm_max.with_structured_output(MultipleQuestionGeneration)

# check_relevance_pipe = check_relevance_chat_template | llm_max_strict | StrOutputParser()
# gen_questions_pipe = gen_questions_chat_template | llm_max | StrOutputParser()

In [ ]:
import json
import time

from tqdm import tqdm

In [ ]:
# def generate_questions(text):
#     while True:
#         try:
#             is_relevant = check_relevance_pipe.invoke({'text': text})
#             try:
#                 is_relevant = json.loads(is_relevant)
#             except:
#                 print('bad_1')
#                 return
            
#             if is_relevant['response'] == 'Нет' or is_relevant['response'] == 'нет':
#                 return
#             sample_questions = '\n'.join(queries.sample(5)['Вопрос '].tolist())
#             new_question = gen_questions_pipe.invoke({'sample_questions':sample_questions, 'text':text})

#             try:
#                 return text, json.loads(new_question)
#             except:
#                 print('bad_2')
#                 return

#         except:
#             time.sleep(3)

In [ ]:
def generate_questions(text):
    while True:
        try:
            is_relevant = check_relevance_pipe.invoke({'text': text})
            if is_relevant.response.strip(' \n.').lower() == 'нет':
            # if is_relevant.split('').split('</think>')[1].strip(' \n.').lower()  == 'нет':
                return None, None, None
            sample_questions = '\n'.join(queries.sample(5)['Вопрос '].tolist())
            new_questions = gen_questions_pipe.invoke({'sample_questions': sample_questions, 'text': text})
            questions = []
            brief_questions = []
            for question in new_questions.questions:
                questions.append(question.question)
                brief_questions.append(question.brief_question)
        
            return text, questions, brief_questions
        except: 
            print('bad')
            time.sleep(5)


In [ ]:
docs_texts[0]

In [ ]:
generate_questions(docs_texts[1])

In [ ]:
bd.

In [ ]:
len(docs_texts)

In [ ]:
from joblib import Parallel, delayed
import time

result_2 = Parallel(n_jobs=4, prefer="threads")(delayed(generate_questions)(text) for text in tqdm(docs_texts[:500]))

In [ ]:
pickle.dump(result_2, open('res_1.pkl', 'wb'))

In [ ]:
from joblib import Parallel, delayed
import time

result_2 = Parallel(n_jobs=4, prefer="threads")(delayed(generate_questions)(text) for text in tqdm(docs_texts[500:1000]))
pickle.dump(result_2, open('res_2.pkl', 'wb'))


In [ ]:
result_2 = Parallel(n_jobs=4, prefer="threads")(delayed(generate_questions)(text) for text in tqdm(docs_texts[1000:1500]))
pickle.dump(result_2, open('res_3.pkl', 'wb'))


In [ ]:
result_2 = Parallel(n_jobs=4, prefer="threads")(delayed(generate_questions)(text) for text in tqdm(docs_texts[1500:]))
pickle.dump(result_2, open('res_4.pkl', 'wb'))


In [ ]:
text = docs_texts[0]

In [ ]:
check_relevance_pipe

In [ ]:
# is_relevant = check_relevance_pipe.invoke({'text': text})

# # if is_relevant.response == 'Нет' or is_relevant.response == 'нет':
# #     1
# # else:
# sample_questions = '\n'.join(queries.sample(5)['Вопрос '].tolist())
# new_question = gen_questions_pipe.invoke({'sample_questions':sample_questions, 'text':text})




In [ ]:
# np.random.seed(1337)
# questions_docs = []
# for text in tqdm(docs_texts):
#     while True:
#         try:
#             is_relevant = check_relevance_pipe.invoke({'text': text})
#             try:
#                 is_relevant = json.loads(is_relevant)
#             except:
#                 break
            
#             if is_relevant['response'] == 'Нет' or is_relevant['response'] == 'нет':
#                 break
#             sample_questions = '\n'.join(queries.sample(5)['Вопрос '].tolist())
#             new_question = gen_questions_pipe.invoke({'sample_questions':sample_questions, 'text':text})

#             try:
#                 questions_docs.append((text, json.loads(new_question)))
#             except:
#                 break

#             break
#         except:
#             time.sleep(3)

#### Получаем ответы 

In [ ]:
res = []
for i in range(1, 5):
    res += pickle.load(open(f'res_{i}.pkl', 'rb'))

len(res)

In [ ]:
res = [r for r in res if r[0] and len(r[1]) == 3 and len(r[2]) == 3]
len(res)

In [ ]:
context_question = []
for r in res:
    for i in range(3):
        context_question.append((r[0], r[1][i]))
        context_question.append((r[0], r[2][i]))

In [ ]:
STUFF_SYSTEM = """NDA"""


chat_template = ChatPromptTemplate.from_messages([
    ('system', STUFF_SYSTEM),
    ('user', '"""NDA"""')
])

chain = (
    chat_template | 
    llm_max_strict | 
    StrOutputParser()
)

In [ ]:
def generate_answer(doc, question):
    while True:
        try:    
            answer = chain.invoke({'context': doc, 'question': question})            
            return answer, doc, question
        except:
            print('bad')
            time.sleep(5)

In [ ]:
result = Parallel(n_jobs=4, prefer="threads")(delayed(generate_answer)(doc, question) for doc, question in tqdm(context_question[2000:]))


In [ ]:
import pickle


pickle.dump(result, open('answers_2.pkl', 'wb'))

In [ ]:
# for i in range(118):
#     question = json.loads(questions_docs[i][1])['question']
#     doc = questions_docs[i][0]
#     answer = answers[i]
#     print('Question:')
#     print(question)
#     print()
#     print('Context:')
#     display_markdown(doc)
#     print('Answer:')
#     print(answer)
#     print()
#     print('-' * 150)
#     print()
